<a href="https://colab.research.google.com/github/leyli16/HousingPricePrediction/blob/main/Final_project_yixu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from string import ascii_letters
import matplotlib.pyplot as plt
import datetime as dt
import requests
from lxml import html
import math
import json
import re
import os

# 1. County Demographics Data Cleaning and Wrangling


## 1.1 Loading County Demogrphics Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("glozab/county-level-us-demographic-data-1990-2020")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/glozab/county-level-us-demographic-data-1990-2020/versions/1


In [ ]:
files = os.listdir(path)
print("Files in dataset:", files)

Files in dataset: ['county_demographics.csv']


In [ ]:
file_path = os.path.join(path, 'county_demographics.csv')
county_demographics_df = pd.read_csv(file_path)

print(county_demographics_df.head())

   year  fips  population  w_population  b_population  o_population  \
0  1990  1025       27289         15579         11643            35   
1  1990  1031       40293         32869          6950           160   
2  1990  1041       13598         10068          3516            11   
3  1990  1053       35526         24377         10050          1045   
4  1990  1101      209537        119702         87856           415   

   nh_population  hi_population  na_population  male_population  ...  \
0          27196             93              0            13052  ...   
1          39831            462              0            19673  ...   
2          13576             22              0             6421  ...   
3          35378            148              0            17454  ...   
4         207933           1604              0            98854  ...   

   age9_population_ratio  age10_population_ratio  age11_population_ratio  \
0                0.06263                 0.05552                

## 1.2 County Demographics Dataset Cleaning

In [ ]:
county_demographics_df.dtypes

,0
year,int64
fips,int64
population,int64
w_population,int64
b_population,int64
o_population,int64
nh_population,int64
hi_population,int64
na_population,int64
male_population,int64


In [ ]:
print(county_demographics_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97287 entries, 0 to 97286
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     97287 non-null  int64  
 1   fips                     97287 non-null  int64  
 2   population               97287 non-null  int64  
 3   w_population             97287 non-null  int64  
 4   b_population             97287 non-null  int64  
 5   o_population             97287 non-null  int64  
 6   nh_population            97287 non-null  int64  
 7   hi_population            97287 non-null  int64  
 8   na_population            97287 non-null  int64  
 9   male_population          97287 non-null  int64  
 10  female_population        97287 non-null  int64  
 11  age0_population          97287 non-null  int64  
 12  age1_population          97287 non-null  int64  
 13  age2_population          97287 non-null  int64  
 14  age3_population       

In [ ]:
print(county_demographics_df.isna().sum().sort_values(ascending=False))

year                       0
fips                       0
population                 0
w_population               0
b_population               0
o_population               0
nh_population              0
hi_population              0
na_population              0
male_population            0
female_population          0
age0_population            0
age1_population            0
age2_population            0
age3_population            0
age4_population            0
age5_population            0
age6_population            0
age7_population            0
age8_population            0
age9_population            0
age10_population           0
age11_population           0
age12_population           0
age13_population           0
age14_population           0
age15_population           0
age16_population           0
age17_population           0
age18_population           0
w_population_ratio         0
b_population_ratio         0
o_population_ratio         0
nh_population_ratio        0
hi_population_

### Rename Columns for Consistency

In [ ]:
county_demographics_df.columns = county_demographics_df.columns.str.strip().str.lower().str.replace(' ', '_')

 ### Handle Duplicates

In [ ]:
county_demographics_df = county_demographics_df.drop_duplicates()
county_demographics_df = county_demographics_df.dropna()
county_demographics_df.head()

,year,fips,population,w_population,b_population,o_population,nh_population,hi_population,na_population,male_population,...,age9_population_ratio,age10_population_ratio,age11_population_ratio,age12_population_ratio,age13_population_ratio,age14_population_ratio,age15_population_ratio,age16_population_ratio,age17_population_ratio,age18_population_ratio
0,1990,1025,27289,15579,11643,35,27196,93,0,13052,...,0.06263,0.05552,0.04998,0.04482,0.04167,0.03767,0.03324,0.02825,0.01843,0.01319
1,1990,1031,40293,32869,6950,160,39831,462,0,19673,...,0.07455,0.06031,0.05180,0.04894,0.04544,0.04145,0.03430,0.02730,0.01660,0.01258
2,1990,1041,13598,10068,3516,11,13576,22,0,6421,...,0.06582,0.05354,0.04574,0.04765,0.05104,0.05273,0.04633,0.03876,0.02456,0.01839
3,1990,1053,35526,24377,10050,1045,35378,148,0,17454,...,0.06820,0.05694,0.05174,0.04771,0.04712,0.04141,0.03502,0.02911,0.01942,0.01368
4,1990,1101,209537,119702,87856,415,207933,1604,0,98854,...,0.06966,0.05219,0.04392,0.04111,0.03967,0.03757,0.02881,0.02294,0.01483,0.01145


In [ ]:
county_demographics_df.describe()

,year,fips,population,w_population,b_population,o_population,nh_population,hi_population,na_population,male_population,...,age9_population_ratio,age10_population_ratio,age11_population_ratio,age12_population_ratio,age13_population_ratio,age14_population_ratio,age15_population_ratio,age16_population_ratio,age17_population_ratio,age18_population_ratio
count,97287.000000,97287.000000,9.728700e+04,9.728700e+04,9.728700e+04,97287.000000,9.728700e+04,9.728700e+04,97287.0,9.728700e+04,...,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000
mean,2005.009436,30412.859950,9.365957e+04,7.529841e+04,1.252406e+04,1129.919044,8.009754e+04,1.356203e+04,0.0,4.598061e+04,...,0.068098,0.067555,0.064839,0.060595,0.054948,0.048050,0.039687,0.030821,0.021798,0.020099
std,8.942825,15147.842995,3.034699e+05,2.259523e+05,5.716518e+04,4815.437070,2.171562e+05,1.058393e+05,0.0,1.489240e+05,...,0.011292,0.010417,0.011904,0.014153,0.014746,0.013994,0.011871,0.009476,0.007503,0.008808
min,1990.000000,1001.000000,5.500000e+01,2.900000e+01,0.000000e+00,0.000000,4.900000e+01,0.000000e+00,0.0,2.900000e+01,...,0.008400,0.013420,0.009150,0.007710,0.005320,0.004840,0.002570,0.000000,0.000000,0.000000
25%,1997.000000,18183.000000,1.094800e+04,9.225500e+03,9.500000e+01,40.000000,1.011600e+04,1.710000e+02,0.0,5.445000e+03,...,0.059970,0.060400,0.055950,0.048960,0.044160,0.038620,0.032010,0.024680,0.016890,0.014210
50%,2005.000000,29179.000000,2.490200e+04,2.166100e+04,8.000000e+02,136.000000,2.327000e+04,6.350000e+02,0.0,1.231900e+04,...,0.068510,0.067780,0.065460,0.060580,0.052770,0.046150,0.038470,0.029950,0.021000,0.018590
75%,2013.000000,45083.000000,6.347900e+04,5.624700e+04,5.440000e+03,550.000000,5.989000e+04,3.148000e+03,0.0,3.135050e+04,...,0.075770,0.074500,0.073120,0.071100,0.064605,0.055630,0.045940,0.035970,0.025670,0.024350
max,2020.000000,99999.000000,1.009486e+07,7.301940e+06,1.437609e+06,172454.000000,5.508975e+06,4.868179e+06,0.0,4.975410e+06,...,0.160000,0.155960,0.153010,0.258060,0.181820,0.183910,0.183580,0.143400,0.144440,0.195400


## 1.2 Loading Fips Code DataSet




In [ ]:
from google.colab import files
uploaded = files.upload()
fips_df = pd.read_csv('state_and_county_fips_master.csv')
fips_df.head()

Saving state_and_county_fips_master.csv to state_and_county_fips_master.csv


,fips,name,state
0,0,UNITED STATES,NaN
1,1000,ALABAMA,NaN
2,1001,Autauga County,AL
3,1003,Baldwin County,AL
4,1005,Barbour County,AL


In [ ]:
fips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3195 entries, 0 to 3194
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   fips    3195 non-null   int64 
 1   name    3195 non-null   object
 2   state   3143 non-null   object
dtypes: int64(1), object(2)
memory usage: 75.0+ KB


## 1.3 Merging the county_demographics_df with fips_df

In [ ]:
county_demographics_merged_df = county_demographics_df.merge(
    fips_df,
    on='fips',
    how='left'
)
county_demographics_merged_df.head()

,year,fips,population,w_population,b_population,o_population,nh_population,hi_population,na_population,male_population,...,age11_population_ratio,age12_population_ratio,age13_population_ratio,age14_population_ratio,age15_population_ratio,age16_population_ratio,age17_population_ratio,age18_population_ratio,name,state
0,1990,1025,27289,15579,11643,35,27196,93,0,13052,...,0.04998,0.04482,0.04167,0.03767,0.03324,0.02825,0.01843,0.01319,Clarke County,AL
1,1990,1031,40293,32869,6950,160,39831,462,0,19673,...,0.05180,0.04894,0.04544,0.04145,0.03430,0.02730,0.01660,0.01258,Coffee County,AL
2,1990,1041,13598,10068,3516,11,13576,22,0,6421,...,0.04574,0.04765,0.05104,0.05273,0.04633,0.03876,0.02456,0.01839,Crenshaw County,AL
3,1990,1053,35526,24377,10050,1045,35378,148,0,17454,...,0.05174,0.04771,0.04712,0.04141,0.03502,0.02911,0.01942,0.01368,Escambia County,AL
4,1990,1101,209537,119702,87856,415,207933,1604,0,98854,...,0.04392,0.04111,0.03967,0.03757,0.02881,0.02294,0.01483,0.01145,Montgomery County,AL


### Sanity Check After Merging

In [ ]:
print(county_demographics_merged_df.columns)         # See all columns
print(county_demographics_merged_df[['fips', 'name']].head())  # Confirm correct matches
print(county_demographics_merged_df.isna().sum())    # Check for unmatched FIPS

Index(['year', 'fips', 'population', 'w_population', 'b_population',
       'o_population', 'nh_population', 'hi_population', 'na_population',
       'male_population', 'female_population', 'age0_population',
       'age1_population', 'age2_population', 'age3_population',
       'age4_population', 'age5_population', 'age6_population',
       'age7_population', 'age8_population', 'age9_population',
       'age10_population', 'age11_population', 'age12_population',
       'age13_population', 'age14_population', 'age15_population',
       'age16_population', 'age17_population', 'age18_population',
       'w_population_ratio', 'b_population_ratio', 'o_population_ratio',
       'nh_population_ratio', 'hi_population_ratio', 'na_population_ratio',
       'male_population_ratio', 'female_population_ratio',
       'age0_population_ratio', 'age1_population_ratio',
       'age2_population_ratio', 'age3_population_ratio',
       'age4_population_ratio', 'age5_population_ratio',
       'age6_popula

## 1.4 Cleaning the Merged Data

Checking for unmatched FIPS — that output tells us 174 rows in the county_demographics_df didn't find a match in fips_df, because name, state, and any other info from fips_df came back as NaN.

In [ ]:
unmatched = county_demographics_merged_df[county_demographics_merged_df['name'].isna()]
print(unmatched['fips'].unique()[:10])

[ 2910  4910 51917  8911  2201  8912  8913  2280  2232  2010]


In [ ]:
county_demographics_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97287 entries, 0 to 97286
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     97287 non-null  int64  
 1   fips                     97287 non-null  int64  
 2   population               97287 non-null  int64  
 3   w_population             97287 non-null  int64  
 4   b_population             97287 non-null  int64  
 5   o_population             97287 non-null  int64  
 6   nh_population            97287 non-null  int64  
 7   hi_population            97287 non-null  int64  
 8   na_population            97287 non-null  int64  
 9   male_population          97287 non-null  int64  
 10  female_population        97287 non-null  int64  
 11  age0_population          97287 non-null  int64  
 12  age1_population          97287 non-null  int64  
 13  age2_population          97287 non-null  int64  
 14  age3_population       

Drop the unmatched rows.

In [ ]:
county_demographics_merged_df = county_demographics_merged_df[county_demographics_merged_df['name'].notna()].reset_index(drop=True)
county_demographics_merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97113 entries, 0 to 97112
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     97113 non-null  int64  
 1   fips                     97113 non-null  int64  
 2   population               97113 non-null  int64  
 3   w_population             97113 non-null  int64  
 4   b_population             97113 non-null  int64  
 5   o_population             97113 non-null  int64  
 6   nh_population            97113 non-null  int64  
 7   hi_population            97113 non-null  int64  
 8   na_population            97113 non-null  int64  
 9   male_population          97113 non-null  int64  
 10  female_population        97113 non-null  int64  
 11  age0_population          97113 non-null  int64  
 12  age1_population          97113 non-null  int64  
 13  age2_population          97113 non-null  int64  
 14  age3_population       

In [ ]:
county_demographics_merged_df = county_demographics_merged_df.rename(columns={'name': 'county'})
county_demographics_merged_df = county_demographics_merged_df.dropna()
county_demographics_merged_df = county_demographics_merged_df.drop_duplicates()
county_demographics_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97113 entries, 0 to 97112
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     97113 non-null  int64  
 1   fips                     97113 non-null  int64  
 2   population               97113 non-null  int64  
 3   w_population             97113 non-null  int64  
 4   b_population             97113 non-null  int64  
 5   o_population             97113 non-null  int64  
 6   nh_population            97113 non-null  int64  
 7   hi_population            97113 non-null  int64  
 8   na_population            97113 non-null  int64  
 9   male_population          97113 non-null  int64  
 10  female_population        97113 non-null  int64  
 11  age0_population          97113 non-null  int64  
 12  age1_population          97113 non-null  int64  
 13  age2_population          97113 non-null  int64  
 14  age3_population       

In [ ]:
county_demographics_merged_df.head()

,year,fips,population,w_population,b_population,o_population,nh_population,hi_population,na_population,male_population,...,age11_population_ratio,age12_population_ratio,age13_population_ratio,age14_population_ratio,age15_population_ratio,age16_population_ratio,age17_population_ratio,age18_population_ratio,county,state
0,1990,1025,27289,15579,11643,35,27196,93,0,13052,...,0.04998,0.04482,0.04167,0.03767,0.03324,0.02825,0.01843,0.01319,Clarke County,AL
1,1990,1031,40293,32869,6950,160,39831,462,0,19673,...,0.05180,0.04894,0.04544,0.04145,0.03430,0.02730,0.01660,0.01258,Coffee County,AL
2,1990,1041,13598,10068,3516,11,13576,22,0,6421,...,0.04574,0.04765,0.05104,0.05273,0.04633,0.03876,0.02456,0.01839,Crenshaw County,AL
3,1990,1053,35526,24377,10050,1045,35378,148,0,17454,...,0.05174,0.04771,0.04712,0.04141,0.03502,0.02911,0.01942,0.01368,Escambia County,AL
4,1990,1101,209537,119702,87856,415,207933,1604,0,98854,...,0.04392,0.04111,0.03967,0.03757,0.02881,0.02294,0.01483,0.01145,Montgomery County,AL


# 2. FEMA Disaster Declarations Data Cleaning and Wrangling




2.1